In [15]:
from transformers import AutoTokenizer,AutoConfig,LlamaForCausalLM
import torch
import time

In [16]:
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
prompt_text = 'In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity.'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
######## Generate with Full Cache
model = LlamaForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)
# print(model.state_dict())
# checkpoint = copy.deepcopy(model.state_dict())
model.eval().to(torch.device("cuda:0"))


# print("loading checkpoint")
# time.sleep(5)
# config = AutoConfig.from_pretrained(model_path)
# config.heavy_ratio = 0.1
# config.recent_ratio = 0.1
# model_new = convert_kvcache_llama_heavy_recent(model, config)
# model_new.load_state_dict(checkpoint)
# print(model_new.state_dict())
# model_new.eval().to(torch.device("cuda:2"))
# print("loading new model")
# generate_ids_hh = model.generate(input_ids, max_new_tokens=64)
# result_hh = tokenizer.batch_decode(generate_ids_hh, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# print("################## Generated Context with Heavy Hitter Oracle ###################")
# print(result_hh)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [ ]:
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
prompt_text = 'In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity.'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
######## Generate with Full Cache
model = LlamaForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)
# print(model.state_dict())
# checkpoint = copy.deepcopy(model.state_dict())
model.eval().to(torch.device("cuda:2"))
input_ids = tokenizer(prompt_text, add_special_tokens=False, return_tensors='pt').input_ids.to(torch.device("cuda:2"))
input_ids
generate_ids = model.generate(input_ids, max_new_tokens=64, return_dict_in_generate=True)
generate_ids
sq=generate_ids.sequences
kv=generate_ids.past_key_values
kv = torch.save(kv,"/home/KVcache/serverlesskv/Modify_attention_layer/ours/past_key_values1.pt")
out = model.generate(sq,  max_new_tokens=50, use_cache=True,past_key_value = kv, return_dict_in_generate=True)
result = tokenizer.batch_decode(out, skip_special_tokens=True)


In [ ]:
input_ids

In [ ]:
generate_ids = model.generate(input_ids, max_new_tokens=64, return_dict_in_generate=True)

In [ ]:
generate_ids

In [ ]:
sq=generate_ids.sequences

In [ ]:
kv=generate_ids.past_key_values


In [ ]:
out = model.generate(sq,  max_new_tokens=50, use_cache=True,past_key_value = kv, return_dict_in_generate=True)

In [1]:
import torch
from ours.custom_llama import LlamaForCausalLM
# from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

/root/anaconda3/envs/kvcache/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
prompt_text = 'In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity.'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
model = LlamaForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)
model.eval().to(torch.device("cuda:2"))
input_ids = tokenizer(prompt_text, add_special_tokens=False, return_tensors='pt').input_ids.to(torch.device("cuda:2"))
input_ids
generate_ids = model.generate(input_ids, max_new_tokens=64, return_dict_in_generate=True)
sq=generate_ids.sequences
# kv=generate_ids.past_key_values
generate_ids

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]


GenerateDecoderOnlyOutput(sequences=tensor([[  512,   263,  2319, 29892,   289,   504,  1847,  5777,  1725, 17763,
           839,   297,   278,  5192,   310,   263,   325,  4626,   424,  4272,
         29892,   263,   724,   355,   666,   277,   681,  1741, 20220,   287,
         29892, 10124,   263,  1833,   292, 17188,   373,   599,  1058, 16277,
           287,   372, 29889,  1094,   278,  2373, 12628,  3290,  1354,  3262,
          1009, 19785, 12712,   322,  3033,  6751,   297, 17524,  9678,   800,
         29892,   263,  5969, 14927, 11952,  4696,   713,  7802,   278,  5777,
          1725, 29892, 19436,   263, 14826,   287, 11210,   322, 17937,  1218,
           385,   263,  2002,   310,   907, 28157, 29889, 13932, 19066,  7018,
         29892,   540,  4689,   304,   851,   398,   263,  9232,   397,  2738,
           260,  1540, 29892,  4332,   440,  1218,   278, 20026,   411,   670,
         10752,  1319,  7314,   322,  2071, 24455,  2301, 14722,  4034, 29889,
           450, 

In [9]:
result = tokenizer.batch_decode(generate_ids.sequences, skip_special_tokens=True)
print(result)
kv = torch.load("/home/KVcache/serverlesskv/Modify_attention_layer/ours/past_key_values.pt")
kv

['In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity. Without hesitation, he began to strum a melodious tune, captivating the audience with his soulful voice and skilled musicianship. The cafe patrons were entranced, mesmerized by the performance, and the atmosphere was filled with an infectious energy that spread throughout']


In [13]:
print(kv)

None


In [10]:
out = model.generate(sq,  max_new_tokens=50, use_cache=True,past_key_values = kv, return_dict_in_generate=True)
result = tokenizer.batch_decode(out.sequences, skip_special_tokens=True)
result

['In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity. Without hesitation, he began to strum a melodious tune, captivating the audience with his soulful voice and skilled musicianship. The cafe patrons were entranced, mesmerized by the performance, and the atmosphere was filled with an infectious energy that spread throughout the space. As the musician sang, his voice soared, weaving a tapestry of emotions that touched the hearts of everyone present. The lyrics spoke of love, hope, and the beauty of life, and the audience was']